In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (GRU, LSTM, Dense, Dropout,
                                     SimpleRNN, TimeDistributed, Layer)
from tensorflow import expand_dims, reduce_sum, multiply, concat, split, nn, reduce_mean
from tensorflow import print as pr
from numpy import zeros
from tqdm import tqdm

from tensorflow.keras.models import load_model


In [3]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.8 MB 13.3 MB/s 
     |████████████████████████████████| 145 kB 47.8 MB/s 
     |████████████████████████████████| 181 kB 45.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [4]:
def GenCsv():
  File_NAme= "/content/drive/MyDrive/hi/lexicons/hi.translit.sampled."
  for File_Type in ["train", "test", "dev"]:
    fi=open("temp",mode='wt',encoding='utf-8')
    fi.write("Hi\tEn\tLexicons\n")
    FullFile=File_NAme+File_Type +".tsv"
    NewFile=open(FullFile, mode='rt',encoding='utf-8')
    for sent in NewFile.readlines():
      fi.write(sent)
    data=pd.read_table("temp")
    data.to_csv(File_Type+".csv")
GenCsv()

In [5]:
#Data Processing and Encoding in One hot VEctor
def Encoding():
  #Train Test File Name
  Train_File_Name="/content/drive/MyDrive/hi/lexicons/hi.translit.sampled.train.tsv"
  Val_File_Name="/content/drive/MyDrive/hi/lexicons/hi.translit.sampled.dev.tsv"
  Test_File_Name="/content/drive/MyDrive/hi/lexicons/hi.translit.sampled.test.tsv"
  Enc_Tokens=0
  Dec_token=0
  Enc_Data=[]
  Dec_Data=[]
  Dec_Target_Data=[]

  Target=[]
  Input=[]
  input_words=[]
  target_words=[]
  inp_char=set()
  tar_char=set()
  TempFile=open('CombinedData.tsv',mode='wt')
  for filename in [Train_File_Name,Val_File_Name,Test_File_Name]:
    with open(filename,mode='rt') as Current_File:
      data=Current_File.read()
      data=data.split('\n')
      #TempFile.write(data)
      for words in data[:len(data)-1]:
        target,input,x=words.split("\t")
        target = "\t" + target + "\n"
        for char in input:
          inp_char.add(char)
        for char in target:
          tar_char.add(char)
        input_words.append(input)
        target_words.append(target)
     


  inp_char = sorted(list(inp_char)) + ["\n"]
  tar_char = sorted(list(tar_char)) + ["\n"]  
  Enc_Tokens_Count=len(inp_char)
  Dec_Tokens_Count=len(tar_char)
  max_len_word_encoder=max([len(txt) for txt in input_words])
  max_len_word_decoder=max([len(txt) for txt in target_words])

  input_token_index = dict([(char, i) for i, char in enumerate(inp_char)])
  target_token_index = dict([(char, i) for i, char in enumerate(tar_char)])
  Dec_Data=np.zeros((len(input_words),max_len_word_decoder,Dec_Tokens_Count),dtype="float32")
  Dec_Target_Data=np.zeros((len(input_words),max_len_word_decoder,Dec_Tokens_Count),dtype="float32")
  Enc_Data=np.zeros((len(input_words),max_len_word_encoder,Enc_Tokens_Count),dtype="float32")

  for index,(target,input) in enumerate((zip(target_words,input_words))):
    for i,char in enumerate(input):
      Enc_Data[index,i,input_token_index[char]]=1.0
    Enc_Data[index,i+1:,input_token_index["\n"]]=1.0
    for i,char in enumerate(target):
      Dec_Data[index,i,target_token_index[char]]=1.0
      if i>0:
        Dec_Target_Data[index,i-1,target_token_index[char]]=1.0
    Dec_Data[index,i+1:,target_token_index["\n"]]=1.0
    Dec_Target_Data[index,i:,target_token_index["\n"]]=1.0

  File_Length=[]
  Input_Encode={}
  Input_Decode={}
  Target_Decode={}
  Input_Text={}
  Target_Text ={}
  for filename in [Train_File_Name,Val_File_Name,Test_File_Name]:
    with open(filename,mode='rt') as Current_File:
      data=Current_File.readlines()
      File_Length.append(len(data))
  Current=0
  Data_Category=['Train','Val','Test']
  for file_length,data_name in zip(File_Length,Data_Category):
    Input_Encode[data_name]=Enc_Data[Current:Current+file_length,:,:]
    Input_Decode[data_name]=Dec_Data[Current:Current+file_length,:,:]
    Target_Decode[data_name]=Dec_Target_Data[Current:Current+file_length,:,:]
    Input_Text[data_name]=np.array(input_words[Current:Current+file_length])
    Target_Text[data_name]=np.array(target_words[Current:Current+file_length])
    Current+=file_length


  return Input_Encode,Input_Decode,Target_Decode,Input_Text,Target_Text,input_token_index,target_token_index,max_len_word_decoder,Dec_Tokens_Count,Enc_Tokens_Count
      
Input_Encode,Input_Decode,Target_Decode,Input_Text,Target_Text,input_token_index,target_token_index,max_len_word_decoder,Dec_Tokens_Count,Enc_Tokens_Count=Encoding()
print(target_token_index)

{'\t': 0, '\n': 65, 'ँ': 2, 'ं': 3, 'ः': 4, 'अ': 5, 'आ': 6, 'इ': 7, 'ई': 8, 'उ': 9, 'ऊ': 10, 'ऋ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'ङ': 21, 'च': 22, 'छ': 23, 'ज': 24, 'झ': 25, 'ञ': 26, 'ट': 27, 'ठ': 28, 'ड': 29, 'ढ': 30, 'ण': 31, 'त': 32, 'थ': 33, 'द': 34, 'ध': 35, 'न': 36, 'प': 37, 'फ': 38, 'ब': 39, 'भ': 40, 'म': 41, 'य': 42, 'र': 43, 'ल': 44, 'व': 45, 'श': 46, 'ष': 47, 'स': 48, 'ह': 49, '़': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ो': 61, 'ौ': 62, '्': 63, 'ॐ': 64}


In [6]:
#Additive Attention
class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):

        if(len(query.shape) == 2):
            query = expand_dims(query, 1)
            s1 = self.W1(query)
            s2 = self.W2(values)
            score = self.V(nn.tanh(s1 + s2))
            attention_weights = nn.softmax(score, axis=1)
            context_vector = attention_weights * values
            context_vector = reduce_sum(context_vector, axis=1)
            return context_vector, attention_weights
        else:
            query = expand_dims(query, 2)
            values = expand_dims(values, 1)

            s1 = self.W1(query)
            s2 = self.W2(values)
            tsum = s1+s2
            score = self.V(nn.tanh(tsum))
            attention_weights = nn.softmax(score, axis=2)
            context_vector = attention_weights * values
            context_vector = reduce_sum(context_vector, axis=2)
            return context_vector, attention_weights

In [7]:
#Function That Creates Model for Encoder Decoder
def Gen_Model(Layer_Dim,Cell_Type,R_Dropout,dp,IfAttention,Attn_Shape,Encoder_TS,Decoder_TS,Encoder_VS,Decode_VS,Embedding_Size,Encoder_Layers,Decoder_Layers):
  CellType={'LSTM':LSTM,'RNN':SimpleRNN,'GRU':GRU}
  temp=1
  if Cell_Type=='LSTM':
    temp=2
  Layers=len(Layer_Dim)
  Encoder_Inputs=keras.Input(shape=(Encoder_TS,Encoder_VS))
  Decoder_Inputs=keras.Input(shape=(Decoder_TS,Decode_VS))
  last_layer_out=Encoder_Inputs
  Encoder_States=[]
  enc_states=[]
  Enc_Layers=[]

  Encoder_Info = keras.layers.Embedding(input_dim = Enc_Tokens_Count, output_dim = Embedding_Size, input_length = max_len_word_decoder)(Encoder_Inputs)
  Encoder_Layer_Dim = Layer_Dim* Encoder_Layers
  Decoeder_Layer_Dim  = Layer_Dim * Decoder_Layers

  for i in range(Layers):
    Encoder_Info=CellType[Cell_Type](Layer_Dim[i],recurrent_dropout=R_Dropout,dropout=dp,return_state=True,return_sequences=True)
    Encoder_Out=Encoder_Info(last_layer_out)
    enc_states,Encoder_Out=concat(list(Encoder_Out[1:]),axis=-1),Encoder_Out[0]
    last_layer_out=Encoder_Out
    Encoder_States.append(enc_states)
    Enc_Layers.append(Encoder_Info)

  Decoder_States=[]
  Dec_Layers=[]
  last_layer_out=Decoder_Inputs
  for i in range(Layers):
    Decoder_Info=CellType[Cell_Type](Layer_Dim[i],return_state=True,recurrent_dropout=R_Dropout,dropout=dp,return_sequences=True)
    Decoder_Out=Decoder_Info(last_layer_out,initial_state=split(Encoder_States[i],temp, -1))
    dec_states,Decoder_Out=(list(Decoder_Out[1:])),Decoder_Out[0]
    last_layer_out=Decoder_Out
    Decoder_States.append(dec_states)
    Dec_Layers.append(Decoder_Info)

  attn = BahdanauAttention(Attn_Shape)
  attn_context, attn_weights = attn(last_layer_out, Encoder_Out)
  Decoder_dropout=Dropout(dp)
  Decoder_dropout_out=Decoder_dropout(last_layer_out)

  Decoder_Dense=Dense(Decode_VS,activation='softmax')
  Decoder_Time_Distribution=TimeDistributed(Decoder_Dense)
  Decoder_Output=Decoder_Time_Distribution(Decoder_dropout_out)
  Final_Model=Model(inputs=[Encoder_Inputs,Decoder_Inputs],outputs=Decoder_Output)
  
#Work For EncoderModel
  Enc_input=Input(shape=(Encoder_TS, Encoder_VS))
  ENC_states=[]
  Last_layer_Out = Enc_input
  for i in range(Layers):
    ENC_Out=Enc_Layers[i](Last_layer_Out)
    Enc_Lstmstate,ENC_Out=concat(list(ENC_Out[1:]),axis=-1),ENC_Out[0]
    ENC_states.append(Enc_Lstmstate)
    Last_layer_Out = ENC_Out
    Encoder_Model=Model(Enc_input,[ENC_states,Last_layer_Out])
    Dec_Input=Input(shape=(1, Decode_VS))
    Dec_State_Lstm=[Input(shape=(temp*Layer_Dim[i],))for i in range(Layers)]
    Encoder_Out=Input(shape=(Encoder_TS,Layer_Dim[Layers-1]))
    prev_layer_out=Dec_Input
    DEc_States=[]
    for i in range(Layers):
        Decoder_LSTM_OUT=Dec_Layers[i](prev_layer_out,initial_state=split(Dec_State_Lstm[i],temp,axis=-1))
        DecoderL_State,Decoder_LSTM_OUT=concat(list(Decoder_LSTM_OUT[1:]),axis=-1),Decoder_LSTM_OUT[0]
        prev_layer_out=Decoder_LSTM_OUT
        DEc_States.append(DecoderL_State)
    attn_context, attn_weights = attn(prev_layer_out, Encoder_Out)
    Decoder_Dropout=Decoder_dropout(prev_layer_out)
    Dec_Output=TimeDistributed(Decoder_Dense)(Decoder_Dropout)
    Decoder_Model=Model([Dec_Input,Dec_State_Lstm, Encoder_Out],[Dec_Output,DEc_States])
  return Final_Model,Encoder_Model,Decoder_Model

Code BElow

In [8]:
#The Commented Code Will Generate Model
#uncomment The code to execute
"""#Without Sweep
cell_type = 'GRU'
layer_dimension = [512]
attention = False
attention_shape = 256
dropout = 0.2
recurrent_dropout = 0.1
Embedding_Size=1
Encoder_Layers=2
Decoder_Layers=3
full_model,Enc_Model,Dec_Model = Gen_Model(layer_dimension,cell_type,recurrent_dropout,dropout,attention,attention_shape,20, 21, 27, 66,Embedding_Size,Encoder_Layers,Decoder_Layers)
full_model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
full_model.fit(x=[Input_Encode["Train"][:1000000],Input_Decode["Train"][:1000000]],y=Target_Decode["Train"][:1000000],batch_size=128,epochs=10,validation_data=([Input_Encode["Val"],Input_Decode["Val"]],Target_Decode["Val"]))
full_model.save("Model"+"/train")
Enc_Model.save("Model"+"/Enc_Model")
Dec_Model.save("Model"+"/Dec_Model")"""

Epoch 1/10
346/346 [==============================] - 115s 274ms/step - loss: 1.0967 - accuracy: 0.7316 - val_loss: 0.8793 - val_accuracy: 0.7614
Epoch 2/10
346/346 [==============================] - 79s 228ms/step - loss: 0.9106 - accuracy: 0.7526 - val_loss: 0.8198 - val_accuracy: 0.7729
Epoch 3/10
346/346 [==============================] - 79s 228ms/step - loss: 0.8528 - accuracy: 0.7634 - val_loss: 0.7488 - val_accuracy: 0.7868
Epoch 4/10
346/346 [==============================] - 80s 230ms/step - loss: 0.7490 - accuracy: 0.7880 - val_loss: 0.5859 - val_accuracy: 0.8290
Epoch 5/10
346/346 [==============================] - 76s 221ms/step - loss: 0.5911 - accuracy: 0.8271 - val_loss: 0.4097 - val_accuracy: 0.8777
Epoch 6/10
346/346 [==============================] - 75s 217ms/step - loss: 0.4548 - accuracy: 0.8627 - val_loss: 0.3098 - val_accuracy: 0.9055
Epoch 7/10
346/346 [==============================] - 77s 222ms/step - loss: 0.3715 - accuracy: 0.8861 - val_loss: 0.2556 - val_a

INFO:tensorflow:Assets written to: Model/Enc_Model/assets


INFO:tensorflow:Assets written to: Model/Enc_Model/assets


INFO:tensorflow:Assets written to: Model/Dec_Model/assets


INFO:tensorflow:Assets written to: Model/Dec_Model/assets


In [24]:
def decode_sequence(input_seqs, encoder_model, decoder_model):
  #Encoding as state vector
  RTargetIndex=dict((i, char) for char,i in target_token_index.items())
  Value_State, enc_out = encoder_model.predict(input_seqs)
  old_states_value = Value_State[:]
  TargetS=np.zeros((len(input_seqs),1,Dec_Tokens_Count))
  TargetS[:,0,target_token_index['\t']] = 1.
  Stop=False
  Sentence_Decoded=[""]*len(input_seqs)
  while not Stop:
    to_split=decoder_model.predict([TargetS,Value_State,enc_out])
    output_tokens,Value_State, attn_weights=to_split[0],list(to_split[1:-1]),to_split[-1]
    Sampled_Token=np.argmax(output_tokens, axis=-1)
    sampled_chars=[RTargetIndex[Sampled_Token[i][0]] for i in range(0,len(input_seqs))]
    for i in range(0, len(input_seqs)) :
      Sentence_Decoded[i]=Sentence_Decoded[i] + str(sampled_chars[i])
      if len(Sentence_Decoded[0])>max_len_word_decoder:
        Stop = True
      TargetS = np.zeros((len(input_seqs),1,Dec_Tokens_Count))
      for i in range(0,len(input_seqs)):
        TargetS[i,0,Sampled_Token[i]] =1.
  Sentence_Decoded=[seq[0:seq.find('\n')] for seq in Sentence_Decoded]
  #print(Sentence_Decoded)
  return Sentence_Decoded

In [22]:
import numpy as np
from tqdm import tqdm
from wandb.keras import WandbCallback
import wandb
import os

#Main Funciton That Creates Model And Find Predictions over VAlidation Data Set
def Main(config=None):
  with wandb.init(config=config):
    config=wandb.config
    wandb.run.name =f"cell_type_{config.cell_type}_layer_org_{config.layer_dimensions}_drpout_{(config.dropout)}_rec-drpout_{(config.recurrent_dropout)}_bs_{config.batch_size}_opt_{config.optimizer}"

    full_model,inc_model,inf_dec_model=Gen_Model(config.layer_dimensions,config.cell_type,config.recurrent_dropout,config.dropout,config.attention,config.attention_shape,20, 21, 27, 66,config.Embedding_Size,config.Encoder_Layers,config.Decoder_Layers)
    full_model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
    full_model.fit(x=[Input_Encode["Train"][:999999],Input_Decode["Train"][:999999]],y=Target_Decode["Train"][:999999],batch_size=128,epochs=config.epochs,validation_data=([Input_Encode["Val"],Input_Decode["Val"]], Target_Decode["Val"]),callbacks=[WandbCallback(save_model=False)])

    RTargetIndex=dict((i, char) for char,i in target_token_index.items())


    def decode_sequence(input_seqs, encoder_model, decoder_model):
        # Encode the input as state vectors.
        states_value, enc_out = encoder_model.predict(input_seqs)
        old_states_value = states_value[:]
        target_seq = np.zeros((len(input_seqs), 1, Dec_Tokens_Count))
        target_seq[:,0,target_token_index['\t']] = 1.
        stop_condition = False
        decoded_sentence = [""] * len(input_seqs)
        while not stop_condition:
            to_split = decoder_model.predict([target_seq, states_value, enc_out])
            output_tokens,states_value, attn_weights = to_split[0], list(to_split[1:-1]), to_split[-1]
            sampled_token_index = np.argmax(output_tokens, axis = -1)
            sampled_chars = [RTargetIndex[sampled_token_index[i][0]] for i in range(0, len(input_seqs))]
            for i in range(0, len(input_seqs)) :
                decoded_sentence[i] = decoded_sentence[i] + str(sampled_chars[i])
            if len(decoded_sentence[0]) > max_len_word_decoder:
                stop_condition = True
            target_seq = np.zeros((len(input_seqs), 1, Dec_Tokens_Count))
            for i in range(0, len(input_seqs)) :
              target_seq[i, 0, sampled_token_index[i]] = 1.
        decoded_sentence = [seq[0:seq.find('\n')] for seq in decoded_sentence]
        return decoded_sentence

    input_seqs=Input_Encode["Val"]
    target_sents=Target_Text["Val"]
    input_texts=Input_Text["Val"]
    Acc_Val=0
    size_batch=64
    n=len(input_seqs)
    TableLog=[]
    for IndexSq in tqdm(range(0,n,size_batch)):
        input_seq=input_seqs[IndexSq:min(n,IndexSq + size_batch)]
        decoded_sentences=decode_sequence(input_seq, inc_model, inf_dec_model)
        target_sentences=[str(target_sents[i : i + 1][0][1:-1]) for i in range(IndexSq, min(n, IndexSq + size_batch))]
        for i in range(0,len(decoded_sentences)) :
          if(decoded_sentences[i]==target_sentences[i]):
              Acc_Val += 1
        if(IndexSq < size_batch):
            for i in range(IndexSq, min(n, IndexSq +size_batch)) :
              TableLog.append([input_texts[i], decoded_sentences[i-IndexSq],target_sentences[i-IndexSq]])
              print({f"input_{i}": input_texts[i],f"output_{i}":decoded_sentences[i-IndexSq],f"target_{i}":target_sentences[i-IndexSq]})

    wandb.log({"Validation Table":wandb.Table(data=TableLog,columns=["Input", "Decoded", "Target"])})
    Acc_Val/=n
    wandb.log({"val_avg_acc":Acc_Val})


In [23]:
#WandB Functionality
#Use The Sweep Config To change The following Hyperparameters
Sconfig = {'name': 'Test-Sweep', 'method': 'grid'}
# Wandb default config
parameters_dict  = {
    "epochs": {"values":[15]},
    "batch_size": {"values":[128]},
    "layer_dimensions": {"values":[[512]]},
    "cell_type": {"values":['LSTM']},
    "dropout": {"values":[0.2]},
    "recurrent_dropout": {"values":[0.2]},
    "optimizer": {"values":["adam"]},
    "attention": {"values":[False]},
    "attention_shape":{"values":[256]},
    "Embedding_Size":{"values":[32]},
    "Encoder_Layers":{"values":[2]},
    "Decoder_Layers":{"values":[2]},
    "Beam_Width":{"values":[3]},

}
Sconfig['parameters'] = parameters_dict
sweep_id = wandb.sweep(Sconfig, project = 'DL_Assignment3')
wandb.agent(sweep_id, function=Main)

Create sweep with ID: d5fvbhza
Sweep URL: https://wandb.ai/sahilb/DL_Assignment3/sweeps/d5fvbhza


wandb: Agent Starting Run: 1xa0tose with config:
wandb: 	Beam_Width: 3
wandb: 	Decoder_Layers: 2
wandb: 	Embedding_Size: 32
wandb: 	Encoder_Layers: 2
wandb: 	attention: False
wandb: 	attention_shape: 256
wandb: 	batch_size: 128
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	epochs: 15
wandb: 	layer_dimensions: [512]
wandb: 	optimizer: adam
wandb: 	recurrent_dropout: 0.2


Epoch 1/15
333/346 [===========================>..] - ETA: 3s - loss: 1.1070 - accuracy: 0.7296

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
#Best Model Predictions
with wandb.init():
  wandb.run.name = f"Best_Model"
  Enc_Model =tf.keras.models.load_model("Model"+"/Enc_Model")
  Dec_Model =tf.keras.models.load_model("Model"+"/Dec_Model")
  input_seqs = Input_Encode["Test"]
  target_sents=Target_Text["Test"]
  input_texts=Input_Text["Test"]
  n=len(input_seqs)
  log_table=[]
  test_acc=0
  BATCH_SIZE=64

  predictions_vanilla = open("Predictions"+"Best_Model", 'w')
  for seq_index in tqdm(range(0,n,BATCH_SIZE)):
      input_seq = input_seqs[seq_index:min(n,seq_index+BATCH_SIZE)]
      Decoded_Data = decode_sequence(input_seq,Enc_Model, Dec_Model)
      Target_Data=[str(target_sents[i : i + 1][0][1:-1]) for i in range(seq_index, min(n, seq_index + BATCH_SIZE))]
      edit_distances=[]
      for i in range(0,len(Decoded_Data)) :
        if(Decoded_Data[i]==Target_Data[i]):
            test_acc+=1
      for i in range(seq_index,min(n,seq_index+BATCH_SIZE)) :
          predictions_vanilla.write(input_texts[i] +" | "+Decoded_Data[i -seq_index] + " | " +Target_Data[i-seq_index] +'\n')
      if(seq_index < BATCH_SIZE):
          for i in range(seq_index,min(n, seq_index + BATCH_SIZE)) :
            log_table.append([input_texts[i],Decoded_Data[i-seq_index], Target_Data[i-seq_index]])
            print({f"input_{i}":input_texts[i],f"output_{i}": Decoded_Data[i-seq_index],f"target_{i}":Target_Data[i-seq_index]})

  wandb.log({"Validation Table":wandb.Table(data=log_table,columns=["Input", "Prediction", "Target"])})
  print("Test accuracy is : ", test_acc/n)
  wandb.log({"Test accuracy":test_acc/n})